In [15]:
import pandas as pd
import numpy as np
import os
import glob
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from scipy.stats import skew, entropy
from random import shuffle

- За даними акселерометра з мобільного телефону потрібно класифікувати, якою діяльністю займається людина: йде, стоїть, біжить чи йде по сходах. 

1. Збір даних
   
Я зроблю одну вибірку - сиру дату (все разом), а іншу - "часові" параметри зі [статті](https://drive.google.com/file/d/1-18YEmp0YjV3hN9iI8J1i_FWd55HFwOK/view), cт. 526, Table 3.

In [16]:
def get_raw_data():
    # Base directory where the folders 'running', 'idle', 'walking', 'stairs' are located
    base_dir = "data"

    # Initialize a list to store the data from each file
    data_list = []

    # Loop through each activity folder
    for folder in ["running", "idle", "walking", "stairs"]:
        # Construct the path to the folder
        folder_path = os.path.join(base_dir, folder)

        # Find all CSV files in the folder
        for file in glob.glob(os.path.join(folder_path, "*.csv")):
            # Load the CSV file
            df = pd.read_csv(file)

            # Add an 'activity' column to the DataFrame with the folder name as the label
            df["activity"] = folder

            # Append the DataFrame to the list
            data_list.append(df)

    # Я так бачу
    shuffle(data_list)

    # Combine into a single DataFrame
    data = pd.concat(data_list, ignore_index=True)
    return data


get_raw_data().head()  # .size

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
0,7.675811,-12.167333,0.306458,running
1,6.804322,13.364434,11.113884,running
2,39.188293,26.063280,13.656527,running
3,4.453215,-3.208230,-5.602432,running
4,-0.866701,6.550536,-4.941632,running


In [17]:
def get_features(df: pd.DataFrame) -> dict:

    features = {
        "max_value_x": df["accelerometer_X"].max(),
        "min_value_x": df["accelerometer_X"].min(),
        "entropy_x": entropy(df["accelerometer_X"].value_counts(normalize=True)),
        "iqr_x": np.subtract(*np.percentile(df["accelerometer_X"], [75, 25])),
        "max_value_y": df["accelerometer_Y"].max(),
        "min_value_index_y": df[
            "accelerometer_Y"
        ].idxmin(),  # Index of minimum value
        "mad_y": np.mean(
            np.abs(df["accelerometer_Y"] - df["accelerometer_Y"].mean())
        ),  # Manually calculated Mean absolute deviation - .mad() doesn't work on series
        "median_y": df["accelerometer_Y"].median(),
        "skewness_y": skew(df["accelerometer_Y"]),
        "std_y": df["accelerometer_Y"].std(),
        "rmse_y": np.sqrt(
            np.mean(df["accelerometer_Y"] ** 2)
        ),  # Root mean square error
        "skewness_z": skew(df["accelerometer_Z"]),
    }

    return features

In [18]:
def get_time_domain_features_data():
    # Base directory where the folders 'running', 'idle', 'walking', 'stairs' are located
    base_dir = "data"

    # Initialize a list to store the data from each file
    data_list = []

    # Loop through each activity folder
    for folder in ["running", "idle", "walking", "stairs"]:
        # Construct the path to the folder
        folder_path = os.path.join(base_dir, folder)

        # Find all CSV files in the folder
        for file in glob.glob(os.path.join(folder_path, "*.csv")):
            # Load the CSV file
            df = pd.read_csv(file)
            df = get_features(df)

            # Label
            df["activity"] = folder

            # Append to the list
            data_list.append(df)

    # Я так бачу
    shuffle(data_list)

    # Combine into a single DataFrame
    data = pd.DataFrame(data_list)
    return data

get_time_domain_features_data().head()

,max_value_x,min_value_x,entropy_x,iqr_x,max_value_y,min_value_index_y,mad_y,median_y,skewness_y,std_y,rmse_y,skewness_z,activity
0,1.240197,-13.962984,3.401197,5.146336,-0.344765,15,4.101810,-7.924807,-0.151133,4.851926,10.269361,3.076586,walking
1,34.567486,-5.679046,3.401197,9.575610,33.681630,5,11.371820,7.347805,0.040536,14.444148,16.008699,0.806467,running
2,33.073500,-5.133168,3.401197,9.543288,33.202790,28,12.686289,7.228096,-0.158042,15.826460,17.298391,-0.465914,running
3,18.904617,-8.015787,3.401197,10.350134,28.878860,7,7.454608,8.449137,-0.008434,9.764702,12.627250,0.998510,running
4,26.499025,-10.994174,3.401197,15.861586,27.159824,8,9.854960,9.160215,-0.617647,12.792557,14.521282,0.121143,running


2. Аналіз даних 

Надрукуємо репорти для SVM та випадкового лісу для моїх вибірок і проаналізуємо результати. 

In [19]:
def class_report(feature_df, flag = ''):
    X = feature_df.drop("activity", axis=1)
    y = feature_df["activity"]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    svm_clf = make_pipeline(StandardScaler(), SVC(gamma="auto"))
    svm_clf.fit(X_train, y_train)

    rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_clf.fit(X_train, y_train)

    svm_predictions = svm_clf.predict(X_test)
    print(f"\n{flag} SVM Classification Report:")
    print(classification_report(y_test, svm_predictions))

    rf_predictions = rf_clf.predict(X_test)
    print(f"\n{flag} Random Forest Classification Report:")
    print(classification_report(y_test, rf_predictions))

In [20]:
feature_dfs = [
    get_raw_data(),
    get_time_domain_features_data(),
]

flags = [
    'RAW DATA',
    'TIME FEATURES',
]

for dataset, flag in zip(feature_dfs, flags):
    class_report(dataset, flag=flag)


RAW DATA SVM Classification Report:
              precision    recall  f1-score   support

        idle       0.96      0.99      0.97      6192
     running       0.93      0.91      0.92     20488
      stairs       1.00      0.00      0.00       927
     walking       0.81      0.90      0.85     11165

    accuracy                           0.90     38772
   macro avg       0.92      0.70      0.69     38772
weighted avg       0.90      0.90      0.89     38772


RAW DATA Random Forest Classification Report:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      6192
     running       1.00      1.00      1.00     20488
      stairs       1.00      0.99      0.99       927
     walking       1.00      1.00      1.00     11165

    accuracy                           1.00     38772
   macro avg       1.00      1.00      1.00     38772
weighted avg       1.00      1.00      1.00     38772


TIME FEATURES SVM Classification Report:
    

## Порівняння моделей на необроблених даних RAW DATA:
### Продуктивність SVM:

Модель SVM показує хорошу точність і запам’ятовування для класів «неактивний», «біг» і «ходьба», але погано працює в класі «сходи» з відкликанням 0,00, що вказує на те, що вона не змогла правильно ідентифікувати жодні екземпляри «сходів».
Загальна точність становить 90%, із середньозваженим показником F1 89%. Однак macro avg F1 значно нижчий і становить 69%, насамперед через низьку продуктивність у класі «сходи».

### Продуктивність випадкового лісу:

Модель Random Forest демонструє ідеальну або майже ідеальну продуктивність у всіх класах, з точністю та recall 1,00 для більшості класів, 0,99 для «сходів».
Загальна точність і всі середні показники F1 становлять 100%, що вказує на кращу продуктивність ніж SVM для необроблених даних.

## Порівняння моделей за TIME FEATURES:
### Продуктивність SVM:

Ефективність значно покращується завдяки "часовим характеристикам", особливо для класу «сходи», який тепер має показник відкликання 0,47 і F1-оцінку 0,62, хоча це все ще відносно низько порівняно з іншими класами.
Загальна точність становить 99%, із середньозваженим показником F1 98%. macro avg F1 покращується до 90%.

### Продуктивність випадкового лісу:

Подібно до необроблених даних, Random Forest надзвичайно добре працює з "часовими характеристиками", показуючи ідеальні або майже ідеальні оцінки за всіма показниками та класами.
Загальна точність і всі середні оцінки F1 становлять 100%, сподіваюсь це не оверфіттінг.